In [2]:
import pandas as pd

ImportError: cannot import name hashtable

### Eerst gaan we alle datafiles die we nodig hebben mergen tot 1 dataframe

#### 1. Alles van Account mergen

##### Account mergen met activiteit tabellen

In [ ]:
data_acc = pd.read_csv('../data_clean/Account_fixed.csv', sep=',')
data_acti = pd.read_csv('../data_clean/Activiteitscode_fixed.csv', sep=",")
data_acc_acti = pd.read_csv('../data_clean/Account_activiteitscode_fixed.csv', sep=",")

data_acc = data_acc[data_acc['account_reden_van_status'] != 'Inactief']
data_acc = data_acc[data_acc['account_reden_van_status'] != 'Stopzetting']
data_acc = data_acc[data_acc['account_adres_provincie'] == 'Oost-Vlaanderen']

# merge account_activiteitscode with activiteitscode
merged_relation = data_acc_acti.merge(data_acti, left_on='account_activiteitscode_activiteitscode', right_on='activiteitscode_activiteitscode_id', how='inner')
# merge account with the merged relation from above
accounts_merged = data_acc.merge(merged_relation, left_on='account_account_id', right_on='account_activiteitscode_account', how='inner')

accounts_merged = accounts_merged[accounts_merged['activiteitscode_status'] == 'Actief']

In [ ]:
# RAM vrijmaken
try:
    del data_acc
    del data_acti
    del data_acc_acti
    del merged_relation
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [ ]:
accounts_merged.shape

(3192, 22)

##### Accounts mergen met financiële data

In [ ]:
# financiele_data = pd.read_csv('../data_clean/Account_financiële_data_fixed.csv', sep=',')
# financiele_data.shape

In [ ]:
# accounts_merged = accounts_merged.merge(financiele_data, left_on='account_account_id', right_on='financieledata_ondernemingid', how='inner')
# accounts_merged.shape

##### Accounts merge met Contactfiches

In [ ]:
contactfiches = pd.read_csv('../data_clean/Contact_fixed.csv', sep=',')
contactfiches = contactfiches[contactfiches['contact_status'] != 'Inactief']
contactfiches['contact_functietitel'] = contactfiches['contact_functietitel'].str.lower().str.replace(r'[^\w\s]', '', regex=True) \
                                                        .str.replace('  ', ' ').str.strip()
contactfiches.shape

(393514, 6)

In [ ]:
accounts_merged = accounts_merged.merge(contactfiches, left_on='account_account_id', right_on='contact_account', how='inner')
accounts_merged.shape

(16836, 28)

##### Accounts mergen met Functie

In [ ]:
contactfiches_functies = pd.read_csv('../data_clean/Contact_functie_fixed.csv', sep=',')
functies = pd.read_csv('../data_clean/Functie_fixed.csv', sep=',')
contactfiches_functies.shape, functies.shape

((589129, 2), (71, 2))

In [ ]:
contactfiches_functies.columns

Index(['contactfunctie_contactpersoon', 'contactfunctie_functie'], dtype='object')

In [ ]:
accounts_merged = accounts_merged.merge(contactfiches_functies, left_on='contact_contactpersoon_id', right_on='contactfunctie_contactpersoon', how='inner')

In [ ]:
accounts_merged = accounts_merged.merge(functies, left_on='contactfunctie_functie', right_on='functie_functie_id', how='inner')
accounts_merged.shape

(21277, 32)

##### Accounts merge met Personen

In [ ]:
df_persoon = pd.read_csv('../data_clean/Persoon_fixed.csv')

df_persoon = df_persoon[df_persoon['persoon_reden_van_status'] == 'Actief']

for col in df_persoon.columns:
    if col.__contains__('persoon_mail_regio'):
        df_persoon.drop(col, axis=1, inplace=True)

df_persoon.drop(['persoon_persoonsnr_', 'persoon_web_login', 'persoon_reden_van_status'], axis=1, inplace=True)

##############################################################################################################
#### ACCOUNT CONTACT PERSOON MERGE ####
##############################################################################################################

accounts_merged = accounts_merged.merge(df_persoon, left_on='contact_persoon_id', right_on='persoon_persoon_id', how='left')

# NaN waarden opvullen
for col in accounts_merged.columns:
    if col.__contains__('persoon_mail_type') or col.__contains__('persoon_mail_thema'):
        accounts_merged[col] = accounts_merged[col].fillna(0)

accounts_merged['persoon_marketingcommunicatie'] = accounts_merged['persoon_marketingcommunicatie'].fillna('-1')
accounts_merged['persoon_marketingcommunicatie'] = accounts_merged['persoon_marketingcommunicatie'] \
                                                            .str.replace('Strikt', '0').str.replace('Flexibel', '1') \
                                                            .str.replace('Uitgeschreven', '-1').str.replace('unknown', '-1').astype(int)

marketing_pressure_cols = [col for col in accounts_merged.columns if col.__contains__('persoon_mail_type') 
                           or col.__contains__('persoon_mail_thema') or col.__contains__('persoon_marketingcommunicatie')]

marketing_pressure_cols = [col for col in accounts_merged.columns if col.__contains__('persoon_mail_type') 
                           or col.__contains__('persoon_mail_thema') or col.__contains__('persoon_marketingcommunicatie')]

accounts_merged['marketing_pressure'] = accounts_merged[marketing_pressure_cols].sum(axis=1)
accounts_merged['marketing_pressure'] = accounts_merged['marketing_pressure'].astype(int)
accounts_merged.drop(marketing_pressure_cols, axis=1, inplace=True)

#### 2. Alles van Afspraken mergen

In [ ]:
# RAM vrijmaken
try:
    # del financiele_data
    del contactfiches
    del contactfiches_functies
    del functies
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [ ]:
afspraak_betreft_account_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_account_cleaned_fixed.csv', sep=',')
afspraak_account_gelinkt_cleaned = pd.read_csv('../data_clean/Afspraak_account_gelinkt_cleaned_fixed.csv', sep=',')
afspraak_betreft_contact_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_contact_cleaned_fixed.csv', sep=',')

In [ ]:
afspraak_betreft_account_cleaned.shape, afspraak_account_gelinkt_cleaned.shape, afspraak_betreft_contact_cleaned.shape

((4876, 7), (2934, 7), (2552, 7))

In [ ]:
afspraak_betreft_account_cleaned = afspraak_betreft_account_cleaned.drop(['afspraak_betreft_account_betreft_id'], axis=1)
afspraak_account_gelinkt_cleaned = afspraak_account_gelinkt_cleaned.drop(['afspraak_account_gelinkt_account'], axis=1)
afspraak_betreft_contact_cleaned = afspraak_betreft_contact_cleaned.drop(['afspraak_betreft_contactfiche_betreft_id'], axis=1)

In [ ]:
afspraak_betreft_account_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_account_gelinkt_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_betreft_contact_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']

In [ ]:
# afspraak_betreft_account_cleaned["afspraak_betreft"] = "account"
# afspraak_account_gelinkt_cleaned["afspraak_betreft"] = "account"
# afspraak_betreft_contact_cleaned["afspraak_betreft"] = "contact"

In [ ]:
afspraken_merged = pd.concat([afspraak_betreft_account_cleaned, afspraak_account_gelinkt_cleaned, afspraak_betreft_contact_cleaned])

In [ ]:
afspraken = pd.read_csv('../data_clean/Afspraak_alle_fixed.csv', sep=',')
afspraken.columns = ['afspraak_afspraak_id']
afspraken.head()

,afspraak_afspraak_id
0,00082DAA-24BF-ED11-83FF-6045BD8952CE
1,00109C77-4D09-EB11-8114-001DD8B72B61
2,0011A5D9-CC9A-EC11-B400-0022488005A7
3,0016532E-14E3-EC11-BB3C-000D3ABCA38B
4,003760C1-13B4-EC11-983F-00224883C880


In [ ]:
afspraken_merged = pd.merge(afspraken, afspraken_merged, on='afspraak_afspraak_id', how='inner')
afspraken_merged.reset_index(drop=True, inplace=True)

In [ ]:
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10362 entries, 0 to 10361
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  10362 non-null  object
 1   afspraak_thema        10362 non-null  object
 2   afspraak_subthema     10362 non-null  object
 3   afspraak_onderwerp    10362 non-null  object
 4   afspraak_eindtijd     10362 non-null  object
 5   afspraak_keyphrases   10362 non-null  object
dtypes: object(6)
memory usage: 485.8+ KB


In [ ]:
# RAM vrijmaken
try:
    del afspraak_betreft_account_cleaned
    del afspraak_account_gelinkt_cleaned
    del afspraak_betreft_contact_cleaned
    del afspraken
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [ ]:
afspraken_merged = afspraken_merged.drop_duplicates(subset=['afspraak_afspraak_id'], keep='first')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7624 entries, 0 to 10360
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  7624 non-null   object
 1   afspraak_thema        7624 non-null   object
 2   afspraak_subthema     7624 non-null   object
 3   afspraak_onderwerp    7624 non-null   object
 4   afspraak_eindtijd     7624 non-null   object
 5   afspraak_keyphrases   7624 non-null   object
dtypes: object(6)
memory usage: 416.9+ KB


In [ ]:
activiteit_vereist_contact = pd.read_csv('../data_clean/Activiteit_vereist_contact_fixed.csv', sep=',')
activiteit_vereist_contact.shape

(4635, 2)

In [ ]:
afspraken_merged = afspraken_merged.merge(activiteit_vereist_contact, left_on='afspraak_afspraak_id', right_on='activiteitvereistcontact_activityid_id', how='inner')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   afspraak_afspraak_id                    4545 non-null   object
 1   afspraak_thema                          4545 non-null   object
 2   afspraak_subthema                       4545 non-null   object
 3   afspraak_onderwerp                      4545 non-null   object
 4   afspraak_eindtijd                       4545 non-null   object
 5   afspraak_keyphrases                     4545 non-null   object
 6   activiteitvereistcontact_activityid_id  4545 non-null   object
 7   activiteitvereistcontact_reqattendee    4545 non-null   object
dtypes: object(8)
memory usage: 284.2+ KB


In [ ]:
afspraken_merged['afspraak_thema'].value_counts()

afspraak_thema
Lidmaatschap                                       3446
Welt 2.0                                            371
Plato                                               313
Internationaal Ondernemen                            74
Digitalisering, IT & Technologie                     71
Innovatie                                            56
Welt 2.0-2023                                        53
Groeien                                              50
Welt                                                 29
Duurzaam Ondernemen                                  22
Familiebedrijven                                     12
Arbeidsmarkt                                         10
Algemeen Management                                   9
Human Resources                                       8
Opvolging en Overname                                 4
Financieel                                            3
Netwerking                                            3
Mobiliteit                       

#### 3. Alles van Campagnes mergen

##### Campagne mergen met Sessie

In [ ]:
campagne = pd.read_csv('../data_clean/Campagne_fixed.csv', sep=',')
campagne.shape

(4092, 11)

In [ ]:
sessie = pd.read_csv('../data_clean/Sessie_fixed.csv', sep=',')
sessie.shape

(9403, 8)

In [ ]:
sessie_themes_grouped = {
  'Bryo': ['Bryo'],
  'Algemeen': ['Familiebedrijven','Opvolging en Overname','Algemeen Management','Human Resources','Algemeen Management - Intern','Bestuurlijke organisaties'],
  'Onderwijs': ['Opleidingen','Persoonlijke vaardigheden','Onderwijs'],
  'Logistiek': ['Logistiek en Transport','Haven','Supply Chain','Retail'],
  'Welt': ['Welt', 'Welt 2.0', 'Welt 2.0-2023'],
  'Ondernemen': ['Starten', 'Internationaal Ondernemen', 'Jong Voka', 'Groeien', 'Stille Kampioenen', 'Samen doorgaan', 'Strategie'],
  'Duurzaamheid': ['Energie', 'Duurzaam Ondernemen', 'Milieu', 'Mobiliteit'],
  'Lidmaatschap': ['Lidmaatschap'],
  'Innovatie en Technologie': ['Innovatie', 'Digitalisering, IT & Technologie'],
  'Netwerking': ['Netwerking'],
  'Economie': ['Arbeidsmarkt', 'Economie', 'Fiscaal', 'Financieel', 'Marketing & Sales', 'Jobkanaal'],
  'Juridisch': ['Bedrijfsjuridisch', 'Juridisch'],
  'Veiligheid en communicatie': ['Communicatie', 'Veiligheid & Preventie', 'Welzijn en gezondheidszorg'],
  'Andere': ['Plato', 'Ruimtelijke ordening en Infrastructuur', 'Regeringsvorming', 'Industrie', 'Aankoop', 'Privé&Vrije tijd', 'Aantrekkelijke regio', 'Coronavirus']
}

def map_thema(thema):
    for category, themes in sessie_themes_grouped.items():
        if thema in themes:
            return category
    return thema

sessie['sessie_thema_naam_'] = sessie['sessie_thema_naam_'].apply(map_thema)

In [ ]:
sessie['sessie_thema_naam_'].value_counts()

sessie_thema_naam_
Algemeen                      2914
Ondernemen                    1185
Economie                      1083
Netwerking                     993
Onderwijs                      886
Innovatie en Technologie       571
Bryo                           533
unknown                        403
Duurzaamheid                   242
Welt                           211
Veiligheid en communicatie     170
Logistiek                      108
Lidmaatschap                    41
Andere                          37
Juridisch                       26
Name: count, dtype: int64

In [ ]:
result = (sessie.assign(sessie_thema_naam_list=sessie['sessie_thema_naam_'].str.split(', '))
               .explode('sessie_thema_naam_list')
               .drop_duplicates()
               .groupby('sessie_campagne')['sessie_thema_naam_list']
               .agg(lambda x: list(set(x)))
               .reset_index()
               .sort_values(by='sessie_thema_naam_list', key=lambda x: x.str.len(), ascending=False))
result

,sessie_campagne,sessie_thema_naam_list
1492,61FE6D67-F088-E811-80F3-001DD8B72B61,"[Juridisch, Duurzaamheid, Algemeen, Innovatie ..."
3209,CFF2BA02-E99C-EB11-811E-001DD8B72B62,"[Duurzaamheid, Economie, Algemeen, Innovatie e..."
3066,C6D72260-E451-EC11-8C62-000D3ABFCF4A,"[Juridisch, Economie, Algemeen, Innovatie en T..."
2414,9D0868DB-952A-EC11-8124-001DD8B72B61,"[Juridisch, Duurzaamheid, Economie, Innovatie ..."
976,416A235F-F209-E911-80FA-001DD8B72B62,"[Duurzaamheid, Economie, Algemeen, Innovatie e..."
...,...,...
1340,5865C940-3455-E511-8718-005056B06EC4,[Onderwijs]
1341,58665379-0E28-E211-B2FD-005056B06EB4,[unknown]
1342,587BE6F8-A255-E711-80E7-001DD8B72B61,[Welt]
1343,588A47E4-6B68-E711-80E8-001DD8B72B61,[Ondernemen]


In [ ]:
unique_categories = set(category for row in result['sessie_thema_naam_list'] for category in row)

for category in unique_categories:
    result[category] = result['sessie_thema_naam_list'].apply(lambda x: int(category in x))

result.drop('sessie_thema_naam_list', axis=1, inplace=True)

In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3951 entries, 1492 to 3950
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   sessie_campagne             3951 non-null   object
 1   Juridisch                   3951 non-null   int64 
 2   Lidmaatschap                3951 non-null   int64 
 3   Duurzaamheid                3951 non-null   int64 
 4   Economie                    3951 non-null   int64 
 5   Veiligheid en communicatie  3951 non-null   int64 
 6   Bryo                        3951 non-null   int64 
 7   Algemeen                    3951 non-null   int64 
 8   Netwerking                  3951 non-null   int64 
 9   Onderwijs                   3951 non-null   int64 
 10  Logistiek                   3951 non-null   int64 
 11  unknown                     3951 non-null   int64 
 12  Innovatie en Technologie    3951 non-null   int64 
 13  Andere                      3951 non-null   int64 

In [ ]:
campagne["campagne_type_campagne"].value_counts()

campagne_type_campagne
Netwerkevenement    1751
Opleiding           1268
Project              565
Infosessie           394
Projectgebonden       99
Campagne              15
Name: count, dtype: int64

In [ ]:
campagnes_merged = campagne.merge(result, left_on='campagne_campagne_id', right_on='sessie_campagne', how='inner')
campagnes_merged.shape

(3951, 27)

#### CDI

In [ ]:
accounts_merged.shape

(21277, 32)

In [ ]:
df_visit = pd.read_csv('../data_clean/CDI_visits_fixed.csv', sep=',')

df_visit.drop(
    ['visit_ip_postcode', 'visit_aangemaakt_op', 'visit_adobe_reader',
    'visit_contact_naam_', 'visit_containssocialprofile', 'visit_ended_on', 'visit_ip_address',
    'visit_ip_organization', 'visit_keywords', 'visit_ip_longitude', 'visit_ip_latitude', 'visit_referrer', 
    'visit_score', 'visit_started_on', 'visit_ip_status', 'visit_visit_id', 'visit_gewijzigd_op',
    'visit_entry_page', 'visit_exit_page', 'visit_ip_company', 'visit_referring_host', 'visit_referrer_type', 
    'visit_bounce', 'visit_duration', 'visit_browser', 'visit_ip_stad', 'visit_ip_land', 'visit_operating_system',
   ], axis=1, inplace=True)

df_visit.drop_duplicates(inplace=True)

df_visit['visit_first_visit'] = df_visit['visit_first_visit'] \
      .str.replace('Ja', '1').str.replace('Nee', '0') \
      .str.replace('unknown', '-1').astype(int)

df_visit['visit_total_pages'] = df_visit['visit_total_pages']\
      .replace('unknown', '-1.0').astype(float)

##############################################################################################################

# Csv files inlezen
df_mailing = pd.read_csv('../data_clean/CDI_mailing_fixed.csv', sep=',')
df_mailing.drop(['mailing_sent_on'], axis=1, inplace=True)
df_mailing.drop_duplicates(inplace=True)
df_click = pd.read_csv('../data_clean/CDI_sent_email_clicks_fixed.csv', sep=',')
df_click.drop(['sentemail_kliks_contact', 'sentemail_kliks_sent_email_id'], axis=1, inplace=True)
df_click.drop_duplicates(inplace=True)

# Merge en drop kolommen
df_mailing_merged = df_mailing.merge(df_click, left_on='mailing_mailing_id', right_on='sentemail_kliks_e_mail_versturen', how='inner')
df_mailing_merged.drop(['sentemail_kliks_e_mail_versturen'], axis=1, inplace=True)

# Voeg een nieuwe kolom toe voor het aantal mails per groep
df_mailing_merged['aantal_mails'] = df_mailing_merged.groupby(
    ['mailing_mailing_id'])['mailing_mailing_id'].transform('count')

# Sentmail_kliks_clicks optellen per mailing
df_mailing_merged = df_mailing_merged.groupby(['mailing_mailing_id', 'mailing_name', 'mailing_onderwerp'], 
                          as_index=False).agg({'sentemail_kliks_clicks': 'sum', 
                                                **{col: 'first' for col in df_mailing_merged.columns if col not in ['sentemail_kliks_clicks']}})

##############################################################################################################

df_visit_mailing = df_visit.merge(df_mailing_merged, left_on='visit_email_send', right_on='mailing_mailing_id', how='left')
df_visit_mailing.drop(['visit_email_send', 'mailing_mailing_id', 'visit_campaign'], axis=1, inplace=True)

# String kolommen cleanen
df_visit_mailing['mailing_name'] = df_visit_mailing['mailing_name'].str.replace('OV-', '') \
                                            .str.replace('OV ', '').str.replace('OV -', '') \
                                            .str.replace('-', ' ').str.replace(r'[^\w\s]', ' ', regex=True) \
                                            .str.replace('  ', ' ').str.strip().str.lower().str.replace('  ', ' ')

df_visit_mailing['mailing_onderwerp'] = df_visit_mailing['mailing_onderwerp'] \
                                            .str.replace('-', ' ').str.replace(r'[^\w\s]', ' ', regex=True) \
                                            .str.replace('  ', ' ').str.strip().str.lower().str.replace('  ', ' ')

df_visit_mailing['sentemail_kliks_clicks'] \
    = df_visit_mailing['sentemail_kliks_clicks'].replace('unknown', '0').astype(int)

In [ ]:
accounts_merged = accounts_merged.merge(df_visit_mailing, left_on='contact_contactpersoon_id', right_on='visit_contact', how='left')
# Numeric en categorical kolommen selecteren
num_cols = accounts_merged.select_dtypes(include=['float64', 'int64']).columns
cat_cols = accounts_merged.select_dtypes(include=['object']).columns

# fill NaN values with -1 for numeric columns
accounts_merged[num_cols] = accounts_merged[num_cols].fillna(-1)
# fill NaN values with 'unknown' for categorical columns
accounts_merged[cat_cols] = accounts_merged[cat_cols].fillna('unknown')

# verander alle int64 en int32 naar int8
int_cols = accounts_merged.select_dtypes(include=['int64', 'int32']).columns
accounts_merged[int_cols] = accounts_merged[int_cols].astype('int8')

In [ ]:
# mail_click_freq berekenen
accounts_merged['mail_click_freq'] = accounts_merged.apply(
    lambda row: 0 if row['aantal_mails'] == -1 else round(row['sentemail_kliks_clicks'] / row['aantal_mails']), axis=1)
accounts_merged.drop(['sentemail_kliks_clicks', 'aantal_mails'], axis=1, inplace=True)

# marketing_pressure berekenen
accounts_merged['marketing_pressure'] = accounts_merged['marketing_pressure'] + accounts_merged['mail_click_freq'] 
+ accounts_merged['visit_total_pages'] + accounts_merged['visit_first_visit']

accounts_merged.drop(['mail_click_freq', 'visit_total_pages', 'visit_first_visit', 'visit_campagne_code', 'visit_contact', 'visit_time', 'mailing_name', 'mailing_onderwerp'], axis=1, inplace=True)
accounts_merged.shape

(31287, 32)

In [ ]:
accounts_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31287 entries, 0 to 31286
Data columns (total 32 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   account_account_id                                         31287 non-null  object
 1   account_adres_geografische_regio                           31287 non-null  object
 2   account_adres_geografische_subregio                        31287 non-null  object
 3   account_adres_plaats                                       31287 non-null  object
 4   account_adres_postcode                                     31287 non-null  object
 5   account_adres_provincie                                    31287 non-null  object
 6   account_industriezone_naam_                                31287 non-null  object
 7   account_is_voka_entiteit                                   31287 non-null  int8  
 8   account_ondernem

#### 4. Accounts mergen met Afspraken

In [ ]:
onnodige_kolommen_accounts = [
#  'account_account_id',
 'account_adres_postcode',
 'account_industriezone_naam_',
 'account_oprichtingsdatum',
 'account_adres_plaats',
 'account_voka_nr_',
 'account_primaire_activiteit',
 'account_is_voka_entiteit',
 'account_status',
 'account_adres_geografische_regio',
 'account_adres_provincie',
 'account_reden_van_status',
 'account_adres_land',
 'account_activiteitscode_account',
 'account_activiteitscode_activiteitscode',
 'account_activiteitscode_inf_account_inf_activiteitscodeid',
 'activiteitscode_activiteitscode_id',
 'contact_account',
 'contact_functietitel',
 'contact_persoon_id',
 'contact_voka_medewerker',
 'contact_status',
#  'financieledata_ondernemingid',
#  'financieledata_gewijzigd_op',
 'contactfunctie_contactpersoon',
 'contactfunctie_functie',
 'functie_functie_id',
 'activiteitscode_status',
 'persoon_persoonsnr_',
 'persoon_web_login',
 'persoon_reden_van_status',
]

accounts_merged = accounts_merged.drop(onnodige_kolommen_accounts, axis=1)

KeyError: "['contact_persoon_id'] not found in axis"

In [ ]:
grouping_categories = {
    'Lidmaatschap': ['Lidmaatschap'],
    'Welt': ['Welt', 'Welt 2.0', 'Welt 2.0-2023'],
    'Plato': ['Plato'],
    'Internationaal Ondernemen': ['Internationaal Ondernemen', 'Internationaal Ondernemen - voor Info en Advies'],
    'Technologie en Innovatie': ['Digitalisering, IT & Technologie', 'Innovatie'],
    'Groeien en Netwerking': ['Groeien', 'Netwerking'],
    'Duurzaamheid': ['Duurzaam Ondernemen', 'Mobiliteit'],
    'Familiebedrijfsmanagement': ['Familiebedrijven', 'Opvolging en Overname'],
    'Arbeidsmarkt': ['Arbeidsmarkt', 'Opleidingen'],
    'Bedrijfsbeheer': ['Algemeen Management', 'Bestuurlijke organisaties', 'Human Resources'],
    'Financieel': ['Financieel', 'Marketing & Sales'],
    'Logistiek en Transport': ['Logistiek en Transport', 'Haven']
}

def map_thema(thema):
    for category, themes in grouping_categories.items():
        if thema in themes:
            return category
    return thema

afspraken_merged['afspraak_thema_grouped'] = afspraken_merged['afspraak_thema'].apply(map_thema)
afspraken_merged = afspraken_merged.drop(['afspraak_thema', 'afspraak_subthema'], axis=1)

In [ ]:
afspraken_merged = afspraken_merged.drop(['afspraak_afspraak_id', 'afspraak_onderwerp', 'afspraak_eindtijd', 'activiteitvereistcontact_activityid_id', 'afspraak_keyphrases'], axis=1)
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 2 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   activiteitvereistcontact_reqattendee  4545 non-null   object
 1   afspraak_thema_grouped                4545 non-null   object
dtypes: object(2)
memory usage: 71.1+ KB


In [ ]:
afspraakthema_counts = afspraken_merged.groupby('activiteitvereistcontact_reqattendee')['afspraak_thema_grouped'].value_counts().unstack(fill_value=0)
accounts_merged = accounts_merged.merge(afspraakthema_counts, left_on='contact_contactpersoon_id', right_index=True, how='left').fillna(0)
accounts_merged.head()

,account_account_id,account_adres_geografische_subregio,account_ondernemingsaard,account_ondernemingstype,activiteitscode_naam,contact_contactpersoon_id,functie_naam,Arbeidsmarkt,Bedrijfsbeheer,Duurzaamheid,Familiebedrijfsmanagement,Financieel,Groeien en Netwerking,Internationaal Ondernemen,Lidmaatschap,Logistiek en Transport,Plato,Technologie en Innovatie,Welt
0,00002DAC-0A69-E111-B43A-00505680000A,Oudenaarde,Diensten,Bedrijf,Overige industrie & diensten,0A9D6946-EB72-E111-B43A-00505680000A,Contact Lidmaatschap,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0016CAE8-BD68-E111-B43A-00505680000A,Gent,Productie & Diensten,Bedrijf,Overige industrie & diensten,04E88B7A-B56A-E111-B43A-00505680000A,Contact Lidmaatschap,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,002C0FBC-C268-E111-B43A-00505680000A,Dendermonde,Productie & Diensten,Familiebedrijf,Ijzer en staal,E668A6B1-EEEC-EA11-8115-001DD8B72B62,Contact Lidmaatschap,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,003B965B-C268-E111-B43A-00505680000A,Gent,Diensten,Bedrijf,Telecom & IT,2C2F82B2-A239-ED11-9DB1-6045BD8952CE,Contact Lidmaatschap,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,004E24A6-0A69-E111-B43A-00505680000A,Dendermonde,Diensten,Bedrijf,Telecom & IT,41CC555F-0174-E111-B43A-00505680000A,Contact Lidmaatschap,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0


In [ ]:
# RAM vrijmaken
try:
    del afspraken_merged
    del activiteit_vereist_contact
    del result
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

#### 5. Cleaning voor de grote merge

In [ ]:
onnodige_kolommen_campagnes = [
 'campagne_campagne_nr',
 'campagne_einddatum',
 'campagne_naam',
 'campagne_reden_van_status',
 'campagne_naam_in_email',
 'campagne_startdatum',
 'campagne_url_voka_be',
 'campagne_status',
 'sessie_campagne',
]

campagnes_merged = campagnes_merged.drop(onnodige_kolommen_campagnes, axis=1)

In [ ]:
# Drop duplicates
accounts_merged = accounts_merged.drop_duplicates(keep='first')
campagnes_merged = campagnes_merged.drop_duplicates(keep='first')

In [ ]:
accounts_merged.info()

In [ ]:
campagnes_merged.info()

#### 6. Alles mergen tot 1 dataframe met een cross join

In [225]:
campagnes_merged['cross_join_key'] = 1
accounts_merged['cross_join_key'] = 1

In [ ]:
merged_total = accounts_merged.merge(campagnes_merged, on='cross_join_key', how='outer').drop('cross_join_key', axis=1)

In [ ]:
# RAM vrijmaken
try:
    del accounts_merged
    del campagnes_merged
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

#### 7. Target label genereren

In [ ]:
import pandas as pd
inschrijving = pd.read_csv('../data_clean/Inschrijving_fixed.csv', sep=',')
inschrijving.shape

(49175, 8)

In [ ]:
merged_total["ingeschreven"] = 0

for index, row in merged_total.iterrows():
    if ((inschrijving['inschrijving_contactfiche'] == row['contact_contactpersoon_id']) & (inschrijving['inschrijving_campagne'] == row['campagne_campagne_id'])).any():
        merged_total.at[index, 'ingeschreven'] = 1
merged_total["ingeschreven"].value_counts()

In [ ]:
merged_total = merged_total.drop(['contact_contactpersoon_id', 'campagne_campagne_id'], axis=1)
merged_total.info()

In [ ]:
merged_total.shape

In [ ]:
# Save to csv
merged_total.to_csv('../data_clean/AA_REC_SYSTEM_SUPERVISED.csv', index=False)

#### 7. Verdere data cleaning en One/Multi hot encoding

In [47]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

oneHot = OneHotEncoder(sparse=False)

In [ ]:
# Subregio, Ondernemingstype
subregio_1hot = oneHot.fit_transform(accounts_merged[['account_adres_geografische_subregio']])

In [ ]:
# Ondernemingstype
be_ondernemingstype = ce.BinaryEncoder(cols=['account_ondernemingstype'])
encoded_ondernemingstype = be_ondernemingstype.fit_transform(accounts_merged['account_ondernemingstype'])
accounts_merged = accounts_merged.join(encoded_ondernemingstype)

In [ ]:
# Ondernemingsaard
be_primaire_activiteit = ce.BinaryEncoder(cols=['account_primaire_activiteit'])
encoded_primaire_activiteit = be_primaire_activiteit.fit_transform(accounts_merged['account_primaire_activiteit'])
accounts_merged = accounts_merged.join(encoded_primaire_activiteit)

In [63]:
# Ondernemingsaard
diensten_column = []
productie_column = []

for label in accounts_merged["account_ondernemingsaard"]:
    if label == "Diensten":
        diensten_column.append(1)
        productie_column.append(0)
    elif label == "Productie":
        diensten_column.append(0)
        productie_column.append(1)
    elif label == "Productie & Diensten":
        diensten_column.append(1)
        productie_column.append(1)
    elif label == "unknown":
        diensten_column.append(0)
        productie_column.append(0)

ondernemingsaard_multihot = pd.DataFrame({"Diensten": diensten_column, "Productie": productie_column})
accounts_merged = accounts_merged.join(ondernemingsaard_multihot)

In [58]:
# Functie_naam
def transform_role(role):
    if "Verantwoordelijke" in role:
        return "Verantwoordelijke"
    elif "Medewerker" in role:
        return "Medewerker"
    else:
        return role

accounts_merged['functie_naam'] = accounts_merged['functie_naam'].apply(transform_role)
functie_1hot = oneHot.fit_transform(accounts_merged[['functie_naam']])

In [ ]:
# Campagne soort
online_column = []
offline_column = []

for label in campagnes_merged["campagne_soort_campagne"]:
    if label == "Offline":
        online_column.append(0)
        offline_column.append(1)
    elif label == "Online":
        online_column.append(1)
        offline_column.append(0)
    elif label == "On en Offline":
        online_column.append(1)
        offline_column.append(1)
    else:
        online_column.append(0)
        offline_column.append(0)

In [ ]:
# Campagne type
campagne_type_1hot = oneHot.fit_transform(campagnes_merged[['campagne_type_campagne']])

In [ ]:
# Al de one hot encodings samenvoegen
accounts_merged = accounts_merged.join(ondernemingsaard_multihot)
accounts_merged = accounts_merged.join(functie_1hot)
accounts_merged = accounts_merged.join(campagne_type_1hot)

# Kolommen verwijderen
accounts_merged = accounts_merged.drop(['account_adres_geografische_subregio', 'account_ondernemingstype', 'account_primaire_activiteit', 'account_ondernemingsaard', 'functie_naam', 'campagne_soort_campagne', 'campagne_type_campagne'], axis=1)

accounts_merged.info()